# Homework 5: K-means Clustering

Due Tuesday March 5

The OptDigits data from the UCI ML repository will be classified using K-means clustering. Each data instance has 64 attributes (pixels) with a value between 0-16, along with the class label.

Clustering will be evaluated using average mean-square-error, mean- square-separation, mean entropy, and accuracy.

## Experiment 1
K = 10

1) Run 5 times with random initial seeds

    1b) Stop run when clusters stop changing
    
2) Choose run with smallest average MSE

In [106]:
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt
from scipy import stats


In [4]:
# Load Data #64 attributes + 1 class = 65
traindata = pd.read_csv("optdigits.train", header = None)
testdata = pd.read_csv("optdigits.test", header = None)

print(traindata.head())
#testdata.head()
traindata = traindata.values #to numpy matrix
testdata = testdata.values

np.random.seed(0)

   0   1   2   3   4   5   6   7   8   9  ...  55  56  57  58  59  60  61  62  \
0   0   1   6  15  12   1   0   0   0   7 ...   0   0   0   6  14   7   1   0   
1   0   0  10  16   6   0   0   0   0   7 ...   0   0   0  10  16  15   3   0   
2   0   0   8  15  16  13   0   0   0   1 ...   0   0   0   9  14   0   0   0   
3   0   0   0   3  11  16   0   0   0   0 ...   0   0   0   0   1  15   2   0   
4   0   0   5  14   4   0   0   0   0   0 ...   0   0   0   4  12  14   7   0   

   63  64  
0   0   0  
1   0   0  
2   0   7  
3   0   4  
4   0   6  

[5 rows x 65 columns]


In [90]:
def rand_cent(data, k):
    """
    Random datapoint selection for initial centroids
    input:
        data: training data set, to compute length
        k: Number of centroids
    output:
        init_cents: array of k-centroids (k-rows, n_attribute-cols)
    """
    n = len(data) - 1
    init_cents = np.zeros((k, data.shape[1]))
    init_ind = np.random.randint(0, n, k)    #array of random indicies into data, of size k
    for i in range(len(init_ind)):
        init_cents[i] = data[init_ind[i]]
    return init_cents

def cluster(data, cents, k):
    """
    Assigns data to clusters
    input:
        data: training data set, to compute euclidian distance (n x 64, STRIP CLASS at end first!)
        k: Number of centroids
        cents: array (len: k) of centroids (k x 64 matrix)
    output:
        #dists: k columns of euclidian distance between row = data_point, column = centroid
        bins: n- length array mapping data-index to cluster (0 to k-1)
    """
#     dists = [] #k columns of euclidian distance between row = data_point, column = centroid
#     for cent in cents:
#         dist = np.linalg.norm(data - cent, axis = 1)
#         dists = np.concatenate((dists, dist.reshape(-1,1)), axis=1) if dists.size else dist.reshape(-1,1)
#         #alternative, reshape at end?
#         #dists = np.vstack(dists, dist) if dists.size else dist
#     bins = np.argmin(dists, axis=1)
    
    dists = np.zeros((k, len(data))) #(k by n_data) of eucl distance between row = centroid, col = data_point
    for i in range(len(cents)):
        cent = cents[i]
        dist = np.linalg.norm(data - cent, axis = 1) #1D array of length (data)
        dists[i] = dist
    bins = np.argmin(dists, axis=0)
    
    return bins

def update(data, bins, k):
    """
    Creates new centroids by averaging clusters
    input:
        data: training data set, to compute euclidian distance (n x 64, STRIP CLASS at end first!)
        bins: n- length array mapping data-index to cluster (0 to k-1)
        k: Number of centroids
    output:
        new_cents: Array of new centroids (k by ncol(aka n_attributes))
    """
    ncol = data.shape[1] #number of columns in data (Attributes)
    csize = np.zeros(k) #track number of data points in each cluster for mean calculation
    ctotal = np.zeros((k, ncol)) #k rows, each with ncol (64) attributes
    
    for i in range(len(data)):
        ctotal[bins[i]] += data[i]
        csize[bins[i]] += 1
    
    csize[csize == 0] = 1 #for divide by 0 cases
    new_cents = ctotal / csize.reshape(-1,1)
    return new_cents

def run_mult(data, k, n_runs):
    final_cents = np.zeros((k, data.shape[1], n_runs)) #3D array with k by n_attributes by height n_runs
    
    for i in range(n_runs):
        
        new_cents = rand_cent(data , k)
        counter = 0
        while(True):
            cents = new_cents
            bins = cluster(data, cents, k)
        #     print(bins.shape)
        #     print(bins[0:100])
            new_cents = update(data, bins, k)
            counter += 1

            if np.array_equal(cents, new_cents):
                break
            elif counter > 100:
                print("max iterations exceeded, 100")
                break
        print("iterations: " + str(counter))
        final_cents[:,:,i] = cents
    return final_cents

In [89]:
#dists = [] #k-size rows of euclidian distance between row = data_point, column = centroid
#closest = [] #Closest centroid 0:k
k = 10
train = traindata[:,:-1] #without labels

cents_5 = run_mult(train, k, 5)

iterations: 31
iterations: 32
iterations: 26
iterations: 41
iterations: 24


In [105]:
def avg_mse(data, cents, k):
    """
    Average mean square error
    input:
        data: training data set, to compute euclidian distance (n x 64, STRIP CLASS at end first!)
        cents (k by ncol(aka n_attributes))
        k: num clusters
    output:
        ret: avg MSE
    """
    mses = []
    bins = cluster(data, cents, k)
    data = np.concatenate((data, bins.reshape(-1,1)), axis=1) #concat bin column to the right side (65th column)
    
    for i in range(len(cents)):
        if np.sum(cents[i]) == 0:
            continue
        cent_data = data[data[:,64]==i][:,:-1]
        dist = np.linalg.norm(cent_data - cents[i], axis = 1)
        mse = np.mean(dist) #average dist^2
        mses += [mse]
    return np.mean(mses)

def classify(train, test, cents, k):
    """
    Associates each cluster with the most frequent class contained within.
    Assign each test point the class of the nearest cluster.
    input:
        train: training data set, (n x 65, WITH CLASS AT END)
        test: test data, to classify
        cents (k by ncol(aka n_attributes))
        k: num clusters
    output:
        ret: predicted classes of the test_data
    """
    bins = cluster(data, cents, k)
    data = train[:,:-1]
    data = np.concatenate((data, bins.reshape(-1,1)), axis=1) #concat bin column to the right side (65th column)
    modes = np.zeros(k)
    
    for i in range(len(cents)):
        if np.sum(cents[i]) == 0:
            continue
        train_cent = train[data[:,64]==i] #all training data (With label), of cluster = i
        m = stats.mode(train_cent[:,64])
        print("m")
        print(m)
        modes[i] = m
        
    return ret

avg_mses = []
for i in range(cents_5.shape[2]):
    avg_mses += [avg_mse(train, cents_5[:,:,i], k)]

best_ind = np.argmin(avg_mses)
best_cents = cents_5[best_ind]

print("avg mses:")
print(avg_mses)
print("best = "+str(best_ind))

[24.753793031661303, 24.84821835611147, 24.783462509909054, 24.66834439467861, 24.852152705406635]
3


In [79]:
a = np.array([[1,2,3],[2,3,4],[3,4,5]])
print(a)
b = np.array([1,5,2])
print(b)
c = a-b
print(c)
#https://stackoverflow.com/questions/7741878/how-to-apply-numpy-linalg-norm-to-each-row-of-a-matrix
#https://docs.scipy.org/doc/numpy/reference/generated/numpy.concatenate.html
print(np.linalg.norm(c, axis = 1))
print(b.reshape(-1,1))
print(np.concatenate((a, b.reshape(-1,1)), axis=1))
print(np.vstack([a,b]))

#https://stackoverflow.com/questions/22732589/concatenating-empty-array-in-numpy
#e = []
#print(np.concatenate((e,a), axis = 1))

print(np.argmin(a, axis=1))

print(b.shape[0])

a[0] += [10, 0, 10]
print(a)
print(a/np.array([10, 2, 1]).reshape(-1,1))
print(a[:,:-1])

[[1 2 3]
 [2 3 4]
 [3 4 5]]
[1 5 2]
[[ 0 -3  1]
 [ 1 -2  2]
 [ 2 -1  3]]
[3.16227766 3.         3.74165739]
[[1]
 [5]
 [2]]
[[1 2 3 1]
 [2 3 4 5]
 [3 4 5 2]]
[[1 2 3]
 [2 3 4]
 [3 4 5]
 [1 5 2]]
[0 0 0]
3
[[11  2 13]
 [ 2  3  4]
 [ 3  4  5]]
[[1.1 0.2 1.3]
 [1.  1.5 2. ]
 [3.  4.  5. ]]
[[11  2]
 [ 2  3]
 [ 3  4]]


In [98]:
#Array deep copy check
a = np.array([1, 2])
b = a
print(b)
a = np.array([2, 3])
print(b)

b[b==2] = 0
print(b)
#Select rows based on val
c = np.array([[1,2],[2,3],[4,2]])
print(c)
d = c[c[:,1]==2]
print(d)

e = []
e += [1]
e += [2]
print(e)
print(c.shape[0])

[1 2]
[1 2]
[1 0]
[[1 2]
 [2 3]
 [4 2]]
[[1 2]
 [4 2]]
[1, 2]
3
